# Python notebook for post-processing ROI responses.
# Create image stack movie with regions plot.
Assumes folder directory structure:
<pre><code>  IMAGING
    image_stacks
    notebooks
    results
</code></pre>
Execute the code sequentially, one block at a time, using &lt;shift-return&gt;.
#### Initialize.

In [ ]:
import glob
import ipywidgets as widgets
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from skimage import io
from skimage.color import gray2rgb
from skimage.feature import canny
from skimage.util import img_as_ubyte

# global variables
if os.name == "nt":
    FILE_SEP = "\\"
else:
    FILE_SEP = "/"


#### Select an image stack, a results directory and one or more regions.
Select multiple regions using command-click.

In [ ]:
%matplotlib widget

# global variables
image_sel = ""    # the selected image stack
results_sel = ""  # the selected results directory
regions_sel = []  # a list of the region files selected
tag_inds = {}     # empty dictionary of tag indices per results directory
image_fps = 10            # image data frames per second

# create image files widget
image_files = sorted([f.split(FILE_SEP)[-1] for f in glob.glob("../image_stacks/*.tif", recursive=False)], key=str.casefold)
image_widget = widgets.Select(options=image_files, description='Image stack', 
                            disabled=False, layout=widgets.Layout(width='400px'))
# create results directory widget
result_dirs = sorted([f.split(FILE_SEP)[-2] for f in glob.glob("../results/*/", recursive=False)], key=str.casefold)
results_widget = widgets.Select(options=[], description='Results dir', 
                            disabled=False, layout=widgets.Layout(width='400px'))
# create regions widget
regions_widget = widgets.SelectMultiple(options=[], description='Regions', 
                            disabled=False, layout=widgets.Layout(width='400px'))

# create numeric input widgets
s = {'description_width':'200px'} # a default widget style
image_fps_widget = widgets.BoundedIntText(value=image_fps, min=1, max=40, step=1,
                    description='Image stack frames per second', disabled=False, layout={'width':'270px'}, style=s)

# create status widget
status_widget = widgets.HTML(value=' ', description=' ')

# update the results dirs based on the image file selection
def update_result_dirs(*args):
  global tag_inds
  rdirs = []
  tag_inds = {}
  for d in result_dirs:
    if not os.path.exists("../results/" + d + "/image_stacks.txt"):
        continue
    f = open("../results/" + d + "/image_stacks.txt", 'r')
    for idx,l in enumerate(f):
      if l.rstrip() == image_widget.value:
        rdirs.append(d)
        tag_inds.update({d:idx})
        break;
    f.close()
  results_widget.options = rdirs    # populate the results widget
  if not rdirs:                     # clear the widget if no regions
    regions_widget.options = []

# update the regions based on the results directory selection
def update_regions(*args):
  if results_widget.value == None:
    return
  region_files = os.listdir("../results/" + results_widget.value)
  region_files = sorted([f for f in region_files if f.startswith('region') and '.csv' in f and 'AVG' not in f][:-1])
  regions_widget.options = region_files

# widget change callbacks
image_widget.observe(update_result_dirs, 'value')
results_widget.observe(update_regions, 'value')

# display and respond to the widgets
update_result_dirs()
update_regions()
def f(w1, w2, w3, w4, w5):
  global image_sel, results_sel, regions_sel, image_fps
  image_sel = image_widget.value
  image_fps = image_fps_widget.value
  results_sel = results_widget.value
  regions_sel = list(regions_widget.value)
  if not image_sel:
    status_widget.value = "No image stack selected."
  elif not results_sel:
    status_widget.value = "No result directory selected."
  elif not regions_sel:
    status_widget.value = "No region(s) selected."
  else:
    status_widget.value = "Selection OK."
display(widgets.interactive(f, w1=image_widget, w2=results_widget, w3=regions_widget, w4=status_widget, w5=image_fps_widget))

#### Select movie options.
Note: An LUT color reference can be found at:
https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html 

In [ ]:
%matplotlib widget

# global variables
lut = 'coolwarm'    # the stack animation color lookup table
roi_key = True
stack_smoothing = 'low'
stack_regions = True

# create plot(s) ROI key widget
roi_key_widget = widgets.Checkbox(value=roi_key, description='Display ROI color key image',
                 disabled=False, indent=True)
# create lut widget
lut_widget = widgets.Dropdown(options=plt.colormaps(), value=lut, description='Stack LUT', 
             disabled=False, layout=widgets.Layout(width='200px'))
# create stack smoothing widget
smoothing_widget = widgets.Dropdown(options=['none', 'low', 'medium'], value=stack_smoothing, 
             description='Smoothing', disabled=False, layout=widgets.Layout(width='200px'))
# create stack regions widget
display_regions_widget = widgets.Checkbox(value=stack_regions, description='Display regions on stack',
                 disabled=False, indent=True)

# display and respond to the widgets
def f(w1,w2,w3,w4):
  global roi_key, lut, stack_smoothing, stack_regions
  roi_key = roi_key_widget.value
  lut = lut_widget.value 
  stack_smoothing = smoothing_widget.value
  stack_regions = display_regions_widget.value
display(widgets.interactive(f, w1=roi_key_widget, w3=lut_widget, w4=smoothing_widget, w2=display_regions_widget))

#### Generate the movie file.
NOTE: May take a minute or two to run depending on the number of regions selected.

In [ ]:
backend = mpl.get_backend()
mpl.use("Agg")  # don't display the animation
print("Processing frame: ", end = '')

# get the default plot colors
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# get the image stack
image_file = "../image_stacks/" + image_sel
image_bits = 10
A0 = io.imread(image_file)
A = np.float32(A0/(2.0**image_bits))
if stack_smoothing != 'none':
  if stack_smoothing == 'low':
    sc = 1                                  # for moving average over 3 frames
  if stack_smoothing == 'medium':
    sc = 2                                  # for moving average over 5 frames
  for i in range(sc,A.shape[0]-sc):         # calculate moving average
    A[i] = np.mean(A[i-sc:i+sc+1], axis=0)
  for i in range(sc):                       # duplicate beginning and ending frames
    A[i] = A[2]
    A[-(1+i)] = A[-(sc+1)]

# create a regions reference image
Aref = gray2rgb(np.copy(A[0])) # first frame of processed image stack

# apply the lut to the image stack
cmap = plt.get_cmap(lut)
A = cmap(A)

# get the apical mask
M = io.imread(glob.glob("../results/" + results_sel + "/*roi_mask_labelled.tif", recursive=False)[0])

# get the results data and color the region perimeters in the image stack
D = []
for idx,s in enumerate(regions_sel):
  data_file = "../results/" + results_sel + '/' + s
  D.append(np.genfromtxt(data_file, delimiter=','))
  temp = np.where(M==int(s.split('-')[0].split('_')[-1]))
  Aref[temp] = mpl.colors.to_rgb(colors[idx%len(colors)])         # using the same colors as the plot(s)
  if stack_regions:
    temp = np.zeros(A0[0].shape)                                  # this region mask
    temp[np.where(M==int(s.split('-')[0].split('_')[-1]))] = 1.0  # find this region value
    perimeter = canny(img_as_ubyte(temp))                         # create a perimeter for this region
    for im in A:                                                  # draw the perimeter in every image frame
      if roi_key:
        im[perimeter] = cmap(0)                                   # color of minimum value if rio key is shown
      else:
        im[perimeter] = mpl.colors.to_rgba(colors[idx])           # using the same colors as the plot(s)
D = np.array(D)  # convert list to numpy array
tag_ind = tag_inds[results_sel] # get the tag index for this results directory

# setup the figure
fig = plt.figure(constrained_layout=True, figsize = [13,6], dpi=100)
gs = fig.add_gridspec(2,2)
ax0 = fig.add_subplot(gs[:,0])  # the image stack
ax1 = fig.add_subplot(gs[1,1])  # the region plot(s)
ax2 = fig.add_subplot(gs[0,1])  # regions reference image 

# image subplot
ax0.axis('off')
im = ax0.imshow(A[0], norm=None, cmap=lut)
pbtime = ax0.annotate("00.0", (18,A.shape[2]-20), size=14)
fig.colorbar(im, ax=ax0, orientation='horizontal', label='Fluorescence (normalized)', shrink=0.85)

# region plot(s) subplot
ax1.set_xlim(1,10.0*np.ceil(D[0,-1,0]/10.0))     # just use final value in first region
ax1.set_ylim(0,100.0*np.ceil(np.amax(D[:,:,tag_ind+1]/100.0)))  # max value across all regions for tag
ax1.set(ylabel="%F/F0")
ax1.set(xlabel="time (s)")
X = [None] * len(regions_sel)     # list of plot value lists (per region)
Y = [None] * len(regions_sel)     #
lines = [None] * len(regions_sel) # list of plotted lines (per region)
for idx,r in enumerate(D):        # plot the first value in each region
  X[idx] = [r[0,0]]
  Y[idx] = [r[0,tag_ind+1]]
  line, = ax1.plot(X[idx],Y[idx])
  lines[idx] = line

# regions reference image
#ax2.axis('off')
ax2.set_xticks([])
ax2.set_yticks([])
[s.set_visible(False) for s in ax2.spines.values()]
ax2.set(ylabel="region key")

if roi_key:
  ax2.imshow(Aref, norm=None)

def init_func():
  return

# animate the figure
def animation_frame(i):
  print(str(i) + ", ", end = '')
  for idx,r in enumerate(D):    # update region plot(s)
    X[idx].append(r[i,0])
    Y[idx].append(r[i,tag_ind+1])  
    lines[idx].set_xdata(X[idx])
    lines[idx].set_ydata(Y[idx])
  im.set_data(A[i])             # get current image frame
  pbtime.set_text('{:0>4.1f}s'.format(i/float(image_fps)))
  return
animation = FuncAnimation(fig, animation_frame, blit=False, repeat=False, init_func=init_func,
                          frames=D.shape[1], interval=np.int(1000.0*D[0,1,0]))

# save the animation to a movie file
animation.save("../results/" + results_sel + "/apical.mp4")

print("DONE.")
mpl.use(backend) # restore the backend for display
